In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


In [ ]:
# ============================================================
# 1. CONFIG
# ============================================================
DATASET_PATH = "/content/skeleton_dataset"  # normal/ and suspicious/
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS_HEAD = 10
EPOCHS_FINE = 10

# ============================================================
# 2. DATA PIPELINE
# ============================================================
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

# ============================================================
# 3. LOAD PRETRAINED RESNET50V2
# ============================================================
base_model = ResNet50V2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.summary()

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# ============================================================
# 4. CLASSIFICATION HEAD (AND EMBEDDING FOR LSTM)
# ============================================================
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 512-d embedding for LSTM
embedding = Dense(512, activation="relu", name="embedding")(x)
x = Dropout(0.5)(embedding)

# Binary classifier
output = Dense(1, activation="sigmoid", name="classifier")(x)

model = Model(inputs=base_model.input, outputs=output)

# ============================================================
# 5. COMPILE (HEAD TRAINING)
# ============================================================
import tensorflow as tf

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="bin_acc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

# ============================================================
# 6. CALLBACKS
# ============================================================
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True, monitor="val_loss"),
    tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDrive/resnet50v2_best.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

# ============================================================
# 7. TRAIN CLASSIFIER HEAD
# ============================================================
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks
)

# ============================================================
# 8. FINE-TUNING (UNFREEZE TOP RESIDUAL BLOCKS)
# ============================================================
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="bin_acc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_FINE,
    callbacks=callbacks
)

# ============================================================
# 9. SAVE FINAL MODEL FOR LSTM FEATURE EXTRACTION
# ============================================================
model.save("/content/drive/MyDrive/resnet50v2_final.keras")
print(">>>>>>>> Training completed <<<<<<<<<<<")
